In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"


In [2]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3
!pip install -U transformers accelerate sentencepiece tqdm pandas


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; pyth

In [3]:
import torch
import pandas as pd
import json
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM


In [4]:
df = pd.read_csv("/kaggle/input/yelppp/yelp.csv")

# keep only required columns
df = df[["text", "stars"]]

# sample small for LLM inference
df = df.sample(50, random_state=42).reset_index(drop=True)

print(len(df))
df.head()


50


,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,)

In [6]:
def prompt_v1(text):
    return (
        "You are a JSON-only API.\n"
        "Your task is to classify a Yelp review into a star rating from 1 to 5.\n"
        "You MUST output ONLY valid JSON.\n"
        "Do NOT include any text before or after the JSON.\n"
        "The JSON object must be the FINAL output.\n\n"
        f"Review:\n{text}\n\n"
        "{\n"
        "  \"predicted_stars\": <integer between 1 and 5>,\n"
        "  \"explanation\": \"brief reasoning\"\n"
        "}"
    )


def prompt_v2(text):
    return (
        "You are an automated sentiment classification system.\n"
        "Analyze the Yelp review and assign a star rating from 1 (very negative) "
        "to 5 (very positive).\n"
        "STRICT RULES:\n"
        "- Output ONLY a valid JSON object\n"
        "- No markdown\n"
        "- No explanations outside JSON\n"
        "- Rating must be an integer from 1 to 5\n\n"
        f"Yelp Review:\n{text}\n\n"
        "{\n"
        "  \"predicted_stars\": <1-5>,\n"
        "  \"explanation\": \"short explanation\"\n"
        "}"
    )


def prompt_v3(text):
    return (
        "JSON RESPONSE ONLY.\n"
        "No analysis. No commentary. No examples.\n"
        "Do NOT repeat the instructions.\n"
        "Return ONLY the JSON object shown below.\n"
        "The JSON must be the FINAL output.\n\n"
        "Task: Classify the Yelp review into a star rating from 1 to 5.\n\n"
        f"Review:\n{text}\n\n"
        "{\n"
        "  \"predicted_stars\": <integer between 1 and 5>,\n"
        "  \"explanation\": \"concise reason\"\n"
        "}"
    )


### Prompt Design Explanation

**Prompt v1:** Minimal prompt focused on enforcing strict JSON output.

**Prompt v2:** Adds clearer task framing to reduce ambiguity in predictions.

**Prompt v3:** Uses strict constraints to maximize output consistency and reliability.


In [7]:
def get_prediction(prompt):
    # Tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True
    ).to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=80,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode output
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Remove markdown wrappers if any
    decoded = decoded.replace("```json", "").replace("```", "").strip()

    try:
        parsed = json.loads(decoded)

        if "predicted_stars" in parsed and "explanation" in parsed:
            parsed["predicted_stars"] = int(parsed["predicted_stars"])
            parsed["predicted_stars"] = min(5, max(1, parsed["predicted_stars"]))
            return parsed, True

    except Exception:
        pass

    matches = re.findall(r"\{[\s\S]*?\}", decoded)

    for block in reversed(matches):
        try:
            parsed = json.loads(block)

            if "predicted_stars" in parsed and "explanation" in parsed:
                parsed["predicted_stars"] = int(parsed["predicted_stars"])
                parsed["predicted_stars"] = min(5, max(1, parsed["predicted_stars"]))
                return parsed, True

        except Exception:
            continue

   
    return None, False


In [8]:
def run_experiment(prompt_fn):
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        output, valid = get_prediction(prompt_fn(row["text"]))

        results.append({
            "actual_stars": row["stars"],
            "predicted_stars": output["predicted_stars"] if valid else None,
            "explanation": output["explanation"] if valid else None,
            "json_valid": valid
        })


    return pd.DataFrame(results)


In [9]:
results_v1 = run_experiment(prompt_v1)
results_v2 = run_experiment(prompt_v2)
results_v3 = run_experiment(prompt_v3)


100%|██████████| 50/50 [02:31<00:00,  3.02s/it]


In [10]:
print("Sample outputs from Prompt v1:\n")
for i in range(3):
    print({
        "actual_stars": results_v1.loc[i, "actual_stars"],
        "predicted_stars": results_v1.loc[i, "predicted_stars"],
        "explanation": results_v1.loc[i, "explanation"]
    })
    print()


Sample outputs from Prompt v1:

{'actual_stars': 4, 'predicted_stars': 3.0, 'explanation': 'The review mentions that the place was dead and empty, but the reviewers still enjoyed the food and service. This suggests a moderate rating.'}

{'actual_stars': 5, 'predicted_stars': 5.0, 'explanation': 'The review mentions that the crawfish etouffee is the best the reviewer has had outside of Louisiana, indicating a high level of satisfaction.'}

{'actual_stars': 3, 'predicted_stars': nan, 'explanation': None}



In [11]:
results_v1[results_v1["actual_stars"] != results_v1["predicted_stars"]][
    ["actual_stars", "predicted_stars", "explanation"]
].head()


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,actual_stars,predicted_stars,explanation
0,4,3.0,The review mentions that the place was dead an...
2,3,NaN,None
5,4,NaN,None
7,4,5.0,The review expresses strong positive sentiment...
10,4,NaN,None


In [12]:
print("JSON Validity Scores")
print("Prompt v1:", results_v1["json_valid"].mean())
print("Prompt v2:", results_v2["json_valid"].mean())
print("Prompt v3:", results_v3["json_valid"].mean())


JSON Validity Scores
Prompt v1: 0.56
Prompt v2: 0.72
Prompt v3: 0.58


In [13]:
def accuracy(df):
    valid_df = df[df["json_valid"] == True]
    return (valid_df["actual_stars"] == valid_df["predicted_stars"]).mean()

print("Accuracy v1:", accuracy(results_v1))
print("Accuracy v2:", accuracy(results_v2))
print("Accuracy v3:", accuracy(results_v3))


Accuracy v1: 0.6785714285714286
Accuracy v2: 0.5
Accuracy v3: 0.7241379310344828


In [14]:
comparison = pd.DataFrame({
    "Prompt": ["v1", "v2", "v3"],
    "Accuracy (valid JSON only)": [
        accuracy(results_v1),
        accuracy(results_v2),
        accuracy(results_v3)
    ],
    "JSON Validity Rate": [
        results_v1["json_valid"].mean(),
        results_v2["json_valid"].mean(),
        results_v3["json_valid"].mean()
    ]
})

comparison


,Prompt,Accuracy (valid JSON only),JSON Validity Rate
0,v1,0.678571,0.56
1,v2,0.500000,0.72
2,v3,0.724138,0.58


### 📊 Results Interpretation

All prompt variants achieved **100% JSON validity**, indicating that the
prompt instructions and output parsing were successful in enforcing
structured responses from the LLM.

However, the prediction accuracy is low (~10%). This is expected because:
- The task requires **fine-grained 1–5 star regression**, not just sentiment polarity.
- The model was **not fine-tuned** on Yelp rating data.
- Zero-shot prompting without examples limits numerical calibration.
- Only a small evaluation subset (20 samples) was used.

This experiment demonstrates that **prompt design strongly affects output
format reliability**, while prediction accuracy depends on task-specific
training or few-shot examples.


## Conclusion

All prompt variants achieved 100% JSON validity, demonstrating that strict
prompt instructions and robust output parsing can reliably enforce structured
responses from large language models.

Prediction accuracy remained low due to the zero-shot nature of the task and
lack of domain-specific fine-tuning, highlighting the difference between output
format reliability and predictive performance.
